<a href="https://colab.research.google.com/github/manulera/ShareYourCloning_backend/blob/master/RSE_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Get files to import sequences from them


'https://raw.githubusercontent.com/manulera/ShareYourCloning_backend/master/dna_functions.py'